In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
%matplotlib inline

In [55]:
import sys

In [56]:
sys.path.append(".")

In [57]:
from ngm import NGM

## Read in metabric data from deeptype paper

In [58]:
import scipy.io
import numpy as np
import pandas as pd

mat = scipy.io.loadmat('BRCA1View20000.mat')
data = mat['data']
mat = {k:v for k, v in mat.items() if k[0] != '_'}
print(len(mat['targets']))
#mat = {k: pd.Series(v[0]) for k, v in mat.items() if len(v) == 0}
mat = {k: pd.Series(v.flatten()) for k, v in mat.items()}

targets = pd.DataFrame.from_dict(mat).dropna()[['id', 'targets']].astype(int)

2133


In [59]:
data.shape

(20000, 2133)

In [60]:
X = data.T

In [61]:
X.shape

(2133, 20000)

## Read in our adjacency matrix

In [62]:
import pandas as pd
import io
import requests
url="https://metabric.s3-us-west-1.amazonaws.com/cat2vec/cat2vec_adj.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [63]:
c['id'] = c['Patient ID'].apply(lambda x: int(x.split("-")[1]))

In [64]:
c_merged = c.merge(targets, on='id')
c_merged = c_merged.set_index('Patient ID')
c_merged = c_merged[c_merged.index]

In [65]:
c_merged.shape

(494, 494)

In [66]:
c_merged

,MB-0002,MB-0005,MB-0006,MB-0008,MB-0010,MB-0014,MB-0020,MB-0022,MB-0028,MB-0035,...,MB-0885,MB-0891,MB-0893,MB-0895,MB-0897,MB-0899,MB-0901,MB-0904,MB-0906,MB-0907
Patient ID,,,,,,,,,,,,,,,,,,,,,
MB-0002,-1.000000,0.049814,0.294911,0.039187,-0.249322,0.348728,0.211491,0.074908,0.039009,-0.387732,...,0.074976,0.098055,0.481728,-0.627232,-0.322279,0.358629,0.242280,-0.341442,-0.358780,-0.151527
MB-0005,0.049814,-1.000000,-0.724841,-0.828614,0.055709,-0.629857,-0.549421,0.352115,0.405221,0.269612,...,0.342066,0.352254,-0.310700,-0.109071,0.054536,-0.644874,-0.529859,-0.003104,0.316354,-0.105369
MB-0006,0.294911,-0.724841,-1.000000,-0.655585,-0.232813,-0.822487,-0.300468,0.200873,0.301604,0.560098,...,0.228852,0.235086,-0.347182,0.200409,-0.191713,-0.839664,-0.288746,-0.200454,0.633718,-0.104822
MB-0008,0.039187,-0.828614,-0.655585,-1.000000,0.045452,-0.591871,-0.578162,0.348633,0.398693,0.225244,...,0.343301,0.370762,-0.302007,-0.150112,0.067975,-0.614083,-0.554857,0.028634,0.272523,-0.079890
MB-0010,-0.249322,0.055709,-0.232813,0.045452,-1.000000,-0.291201,0.505300,-0.075984,-0.028591,0.318952,...,-0.006957,-0.017840,0.288905,-0.023099,-0.729028,-0.248022,0.512799,-0.730339,0.355880,-0.159456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-0899,0.358629,-0.644874,-0.839664,-0.614083,-0.248022,-0.849860,-0.239179,0.217811,0.340402,0.652758,...,0.243182,0.256174,-0.329317,0.254777,-0.208431,-1.000000,-0.225838,-0.189629,0.685713,-0.122964
MB-0901,0.242280,-0.529859,-0.288746,-0.554857,0.512799,-0.200528,-0.870553,-0.111523,-0.103842,-0.245606,...,-0.151128,-0.122368,-0.695124,0.170607,0.480857,-0.225838,-1.000000,0.451450,-0.212358,0.270035
MB-0904,-0.341442,-0.003104,-0.200454,0.028634,-0.730339,-0.238811,0.476475,-0.087855,-0.068353,0.214131,...,-0.056034,-0.069930,0.282914,-0.109065,-0.744894,-0.189629,0.451450,-1.000000,0.240950,-0.108698


In [67]:
c_merged.index = list([int(c.split("-")[1]) for c in c_merged.index])
c_merged.columns = c_merged.index

In [68]:
c_merged

,2,5,6,8,10,14,20,22,28,35,...,885,891,893,895,897,899,901,904,906,907
2,-1.000000,0.049814,0.294911,0.039187,-0.249322,0.348728,0.211491,0.074908,0.039009,-0.387732,...,0.074976,0.098055,0.481728,-0.627232,-0.322279,0.358629,0.242280,-0.341442,-0.358780,-0.151527
5,0.049814,-1.000000,-0.724841,-0.828614,0.055709,-0.629857,-0.549421,0.352115,0.405221,0.269612,...,0.342066,0.352254,-0.310700,-0.109071,0.054536,-0.644874,-0.529859,-0.003104,0.316354,-0.105369
6,0.294911,-0.724841,-1.000000,-0.655585,-0.232813,-0.822487,-0.300468,0.200873,0.301604,0.560098,...,0.228852,0.235086,-0.347182,0.200409,-0.191713,-0.839664,-0.288746,-0.200454,0.633718,-0.104822
8,0.039187,-0.828614,-0.655585,-1.000000,0.045452,-0.591871,-0.578162,0.348633,0.398693,0.225244,...,0.343301,0.370762,-0.302007,-0.150112,0.067975,-0.614083,-0.554857,0.028634,0.272523,-0.079890
10,-0.249322,0.055709,-0.232813,0.045452,-1.000000,-0.291201,0.505300,-0.075984,-0.028591,0.318952,...,-0.006957,-0.017840,0.288905,-0.023099,-0.729028,-0.248022,0.512799,-0.730339,0.355880,-0.159456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,0.358629,-0.644874,-0.839664,-0.614083,-0.248022,-0.849860,-0.239179,0.217811,0.340402,0.652758,...,0.243182,0.256174,-0.329317,0.254777,-0.208431,-1.000000,-0.225838,-0.189629,0.685713,-0.122964
901,0.242280,-0.529859,-0.288746,-0.554857,0.512799,-0.200528,-0.870553,-0.111523,-0.103842,-0.245606,...,-0.151128,-0.122368,-0.695124,0.170607,0.480857,-0.225838,-1.000000,0.451450,-0.212358,0.270035
904,-0.341442,-0.003104,-0.200454,0.028634,-0.730339,-0.238811,0.476475,-0.087855,-0.068353,0.214131,...,-0.056034,-0.069930,0.282914,-0.109065,-0.744894,-0.189629,0.451450,-1.000000,0.240950,-0.108698
906,-0.358780,0.316354,0.633718,0.272523,0.355880,0.660196,-0.188703,-0.357670,-0.469261,-0.792098,...,-0.431136,-0.419074,-0.091748,-0.147430,0.294370,0.685713,-0.212358,0.240950,-1.000000,0.234893


## Fix things so they aren't weird

In [69]:
mat.keys()

dict_keys(['GII', 'data', 'gene', 'id', 'targets'])

In [70]:
genes = [g[0] for g in mat['gene']]
pd.Series(genes)

0          SCGB2A2
1            MUCL1
2          SCGB1D2
3              PIP
4        LOC648852
           ...    
19995    LOC642453
19996    LOC652100
19997    LOC646050
19998    LOC644912
19999    LOC652294
Length: 20000, dtype: object

In [71]:
X_df = pd.DataFrame(X)
X_df.columns = genes
X_df = X_df.set_index(mat['id'])
X_df

,SCGB2A2,MUCL1,SCGB1D2,PIP,LOC648852,DNAJA2,TFF3,S100P,CPB1,CEACAM6,...,DLEU7,LOC641311,LOC645307,IL1RAP,LOC647149,LOC642453,LOC652100,LOC646050,LOC644912,LOC652294
2080,6.365374,5.610561,5.134799,5.735802,5.855315,5.443985,8.005882,5.615179,6.682096,5.756946,...,5.441128,5.409726,5.420998,5.396492,5.358751,5.424770,5.411055,5.242460,5.197767,5.643537
236,14.330442,7.979139,10.244240,7.138279,9.890661,11.195867,13.511164,10.548596,5.491578,6.805606,...,5.392657,5.213895,5.167652,5.273045,5.287980,5.398061,5.226934,5.396744,5.352158,5.811813
741,14.388331,6.016715,12.164435,5.272691,6.468386,5.265518,13.386034,8.764722,7.331663,11.967719,...,5.328035,5.707537,5.327874,5.549589,5.333051,5.368544,5.296182,5.694022,5.459744,5.269015
380,13.111243,7.251862,8.389121,8.664654,10.378585,5.544646,11.159766,8.263224,5.497906,5.923506,...,5.410289,5.351022,5.009093,5.345468,5.176322,5.326991,5.480259,5.292691,5.242160,5.635111
1446,12.243103,7.293489,10.919581,9.566137,8.503568,11.492583,11.531352,9.818938,9.443134,6.889670,...,5.402880,5.320883,5.487783,5.479967,5.332074,5.273141,5.282872,5.222482,5.186136,5.616094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,7.199176,5.708891,5.672914,14.464282,9.816897,5.604835,13.303832,6.391817,13.010996,8.294409,...,5.398490,5.695733,5.363557,5.394837,5.442947,5.423763,5.558225,5.575000,5.636068,5.454823
858,10.354497,14.464282,7.633244,14.464282,11.294018,5.468214,10.886420,12.260033,5.662372,6.274380,...,5.378907,5.537254,5.613097,5.254552,5.205904,5.486546,5.206480,5.161718,5.451296,5.215587
698,9.294401,13.703049,6.063617,10.063176,5.393440,5.529584,10.937517,12.220400,9.985646,8.370227,...,5.554940,5.520596,5.572326,5.317225,5.178158,5.316757,5.210332,5.239428,5.395402,5.501265
1626,14.464282,13.203708,14.169804,13.731721,12.478907,5.347654,14.464282,9.593148,6.165212,11.207149,...,5.470530,5.650387,5.359733,5.374522,5.245155,5.453742,5.353135,5.235716,5.155543,5.442698


In [72]:
y = targets.set_index('id')['targets'].astype(int) # Richa: I wanted to change this to string but it broke pytroch
y = y - y.min() # make sure 0 is the first class. Richa: let's fix this as well
y.value_counts()

2    721
3    491
0    330
1    239
4    202
5    150
Name: targets, dtype: int64

In [73]:
A=c_merged

## Put data into the pytorch format

In [74]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [75]:
c_merged

,2,5,6,8,10,14,20,22,28,35,...,885,891,893,895,897,899,901,904,906,907
2,-1.000000,0.049814,0.294911,0.039187,-0.249322,0.348728,0.211491,0.074908,0.039009,-0.387732,...,0.074976,0.098055,0.481728,-0.627232,-0.322279,0.358629,0.242280,-0.341442,-0.358780,-0.151527
5,0.049814,-1.000000,-0.724841,-0.828614,0.055709,-0.629857,-0.549421,0.352115,0.405221,0.269612,...,0.342066,0.352254,-0.310700,-0.109071,0.054536,-0.644874,-0.529859,-0.003104,0.316354,-0.105369
6,0.294911,-0.724841,-1.000000,-0.655585,-0.232813,-0.822487,-0.300468,0.200873,0.301604,0.560098,...,0.228852,0.235086,-0.347182,0.200409,-0.191713,-0.839664,-0.288746,-0.200454,0.633718,-0.104822
8,0.039187,-0.828614,-0.655585,-1.000000,0.045452,-0.591871,-0.578162,0.348633,0.398693,0.225244,...,0.343301,0.370762,-0.302007,-0.150112,0.067975,-0.614083,-0.554857,0.028634,0.272523,-0.079890
10,-0.249322,0.055709,-0.232813,0.045452,-1.000000,-0.291201,0.505300,-0.075984,-0.028591,0.318952,...,-0.006957,-0.017840,0.288905,-0.023099,-0.729028,-0.248022,0.512799,-0.730339,0.355880,-0.159456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,0.358629,-0.644874,-0.839664,-0.614083,-0.248022,-0.849860,-0.239179,0.217811,0.340402,0.652758,...,0.243182,0.256174,-0.329317,0.254777,-0.208431,-1.000000,-0.225838,-0.189629,0.685713,-0.122964
901,0.242280,-0.529859,-0.288746,-0.554857,0.512799,-0.200528,-0.870553,-0.111523,-0.103842,-0.245606,...,-0.151128,-0.122368,-0.695124,0.170607,0.480857,-0.225838,-1.000000,0.451450,-0.212358,0.270035
904,-0.341442,-0.003104,-0.200454,0.028634,-0.730339,-0.238811,0.476475,-0.087855,-0.068353,0.214131,...,-0.056034,-0.069930,0.282914,-0.109065,-0.744894,-0.189629,0.451450,-1.000000,0.240950,-0.108698
906,-0.358780,0.316354,0.633718,0.272523,0.355880,0.660196,-0.188703,-0.357670,-0.469261,-0.792098,...,-0.431136,-0.419074,-0.091748,-0.147430,0.294370,0.685713,-0.212358,0.240950,-1.000000,0.234893


In [76]:
X_df.loc[A.index]

,SCGB2A2,MUCL1,SCGB1D2,PIP,LOC648852,DNAJA2,TFF3,S100P,CPB1,CEACAM6,...,DLEU7,LOC641311,LOC645307,IL1RAP,LOC647149,LOC642453,LOC652100,LOC646050,LOC644912,LOC652294
2,14.385607,9.517970,11.356071,9.779818,6.025374,5.145908,12.758143,6.343210,6.060934,6.590413,...,5.359574,5.535941,5.471588,5.414016,5.272193,5.231418,5.347781,5.510813,5.323438,5.408310
5,13.607742,14.453875,12.545960,10.412785,8.357205,5.346364,10.074419,11.888042,5.794987,8.598299,...,5.142888,5.495082,5.297388,5.607352,5.280660,5.477391,5.411121,5.427694,5.345672,5.495517
6,14.368799,13.673316,13.768779,13.630298,9.552431,4.849943,13.736075,11.057283,5.355272,8.167450,...,5.635371,5.339480,5.331480,5.460581,5.273268,5.311268,5.512539,5.398603,5.251455,5.657303
8,10.551201,10.569321,6.141232,9.014560,7.947275,10.571770,11.520244,8.616925,6.579840,7.200090,...,5.272019,5.397836,5.154092,5.477319,5.138264,5.535068,5.329249,5.279133,5.373499,5.483210
10,8.660004,5.864028,6.313806,7.799573,5.828219,5.334715,13.096777,6.259485,5.439265,7.894034,...,5.485612,5.609150,5.303219,5.410968,5.325298,5.254830,5.097276,5.622131,5.177518,5.616387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,6.002175,5.371465,5.279233,9.689527,12.887901,5.385932,13.922840,6.604240,5.317769,9.759798,...,5.359749,5.398379,5.224105,5.400004,5.092458,5.420621,5.439776,5.498303,5.309821,5.482541
901,8.583856,5.447145,5.772956,5.323828,5.250099,5.348259,5.318004,8.731500,5.548148,6.206010,...,5.141492,5.578205,5.153203,5.966415,5.268241,5.385566,5.257082,5.361961,5.574043,5.399133
904,5.782602,8.146593,5.372823,7.853609,5.911988,5.402202,7.610225,8.686100,5.242251,5.660677,...,5.618216,5.696409,5.254935,5.502287,5.239574,5.308076,5.373970,5.426283,5.493153,5.729592
906,12.846634,11.959262,10.749467,12.077697,9.685134,5.182048,13.230864,8.296759,6.546391,6.472869,...,5.509244,5.320191,5.241778,5.430578,5.381166,5.293498,5.463120,5.442886,5.236143,5.385670


In [77]:
y.loc[A.index]

2      4
5      3
6      1
8      2
10     0
      ..
899    3
901    0
904    0
906    2
907    0
Name: targets, Length: 494, dtype: int64

In [78]:
edges = c_merged
edges.values[[np.arange(len(edges))]*2] = np.nan
edges = edges.stack().reset_index()
edges

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


,level_0,level_1,0
0,2,5,0.049814
1,2,6,0.294911
2,2,8,0.039187
3,2,10,-0.249322
4,2,14,0.348728
...,...,...,...
243537,907,897,-0.129545
243538,907,899,-0.122964
243539,907,901,0.270035
243540,907,904,-0.108698


In [79]:
batch_size = 6  # The size of input data took for one iteration

## Now construct the model

In [80]:
input_size = X_df.shape[1]
hidden_size = 50 # The number of nodes at the hidden layer
num_classes = len(y.unique())      # The number of output classes. In this case, from 1 to 39

In [81]:
model = NGM.Net(input_size, hidden_size, num_classes)

## Train the model

In [82]:
learning_rate = 0.001  # The speed of convergence
num_epochs=30
training_size = 1000

In [83]:
NGM.train(model, training_size,learning_rate,num_epochs,batch_size, c_merged, X_df, y)

/Users/richagadgil/Desktop/knowledge_driven_modeling/ngm/NGM.py:83: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  edges.values[[np.arange(len(edges))]*2] = np.nan
/Users/richagadgil/Desktop/knowledge_driven_modeling/ngm/NGM.py:125: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  + torch.sum(w_uu * my_softmax_cross_entropy_with_logits(scores_u3, torch.nn.functional.softmax(scores_v3)))


Epoch [2/30], Step [20/166], Loss: -0.3657
Epoch [2/30], Step [40/166], Loss: 2.0310
Epoch [2/30], Step [60/166], Loss: -3.1417
Epoch [2/30], Step [80/166], Loss: 0.6457
Epoch [2/30], Step [100/166], Loss: -3.1755
Epoch [2/30], Step [120/166], Loss: -0.9972
Epoch [2/30], Step [140/166], Loss: -1.3686
Epoch [2/30], Step [160/166], Loss: -0.6412
Epoch [3/30], Step [20/166], Loss: -1.3958
Epoch [3/30], Step [40/166], Loss: -0.3910
Epoch [3/30], Step [60/166], Loss: 0.7083
Epoch [3/30], Step [80/166], Loss: -0.5041
Epoch [3/30], Step [100/166], Loss: 0.3068
Epoch [3/30], Step [120/166], Loss: -2.2460
Epoch [3/30], Step [140/166], Loss: 0.2029
Epoch [3/30], Step [160/166], Loss: -1.0986
Epoch [4/30], Step [20/166], Loss: -0.3512
Epoch [4/30], Step [40/166], Loss: -0.1399
Epoch [4/30], Step [60/166], Loss: -1.9137
Epoch [4/30], Step [80/166], Loss: 0.1571
Epoch [4/30], Step [100/166], Loss: -1.6548
Epoch [4/30], Step [120/166], Loss: -0.6485
Epoch [4/30], Step [140/166], Loss: -2.0905
Epoch 

## Evaluation

In [84]:
from sklearn.metrics import classification_report

In [85]:
ypred,ylabels = NGM.prepare_for_evaluation(model, batch_size, 1000, c_merged, X_df, y)

In [86]:
print(classification_report(ylabels, ypred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.12      1.00      0.21        39
           2       0.00      0.00      0.00       124
           3       0.00      0.00      0.00        95
           4       0.00      0.00      0.00        16

    accuracy                           0.12       334
   macro avg       0.02      0.20      0.04       334
weighted avg       0.01      0.12      0.02       334



//anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [87]:
pd.Series(ypred).value_counts()

1    334
dtype: int64